In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from data.aware_raw import AwareRaw, AwareSpectrogram
from data.cats_and_dogs import Dataset

classes = [
    'Control / healthy / no pulmonary disease',
    'Asthma',
    # 'CF',
    # 'COPD'
]
dataset_raw = AwareRaw("data/AWARE_DATA_LABELS_2023-12-08_1611.csv", "data/id_map.csv", "data/aware_full_1704385505.db", pickle_file="data/aware_segmented.pkl")
dataset = AwareSpectrogram(
    dataset_raw, 
    target_classes=classes, 
    age_balanced=False, 
    output_demogr=False, 
    output_spiro_raw=False, 
    output_spiro_pred=False, 
    output_oscil_raw=False, 
    output_oscil_zscore=False, 
    output_disease_label=True,
    output_inhale_exhale=True,
    relative_change=False, 
    calibration=True, 
    averaged=False, 
    num_channels=3,
    dim_order="BTCHW"
)
imflip = True

# dataset.save_to_pickle('data/aware_spectrogram.pkl')

# dataset = Dataset("data/cats_and_dogs", video=False, dim_order="BTCHW")
# imflip = False

print("Class Distribution:")
print(dataset.class_distribution)
print("Class Weights:")
print(dataset.class_weights)

0it [00:00, ?it/s]

Class Distribution:
[240. 366.]
Class Weights:
[1.2625     0.82786885]


In [4]:
import matplotlib.pyplot as plt
from matplotlib import animation
from torchvision import transforms

trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224), antialias=True),
    transforms.Normalize(0.5, 0.5)
])

df = dataset.data
display(df)

idx = 6
display(df.loc[idx])
ir = df['IR'][idx]
spe = trans(df['Spectrogram'][idx].astype('float32')).flip(dims=(1,))
print(ir.shape)
print(spe.shape)
plt.rcParams["animation.html"] = "jshtml"
fig, ax = plt.subplots(1,2,figsize=(12,5))
line = ax[0].plot(ir[0,...])[0]
ax[0].set_xticks([0,240,480,720,960]) 
ax[0].set_xticklabels([0,5,10,15,20])
ax[0].set_xlabel("Time (ms)")
ax[0].set_ylabel("Amplitude")
ax[0].set_ylim([-0.25,0.25])
im = ax[1].imshow(spe[0,...], cmap='jet')
ax[1].set_xticks([0,56,112,168,224]) 
ax[1].set_xticklabels([0,5,10,15,20])
ax[1].set_yticks([0,56,112,168,224]) 
ax[1].set_yticklabels([12,9,6,3,0])
ax[1].set_xlabel("Time (ms)")
ax[1].set_ylabel("Frequency (kHz)")
text1 = ax[0].text(900, 0.2, '0', fontsize=18)
text2 = ax[1].text(200, 20, '0', fontsize=18, color='white')
def update(frame):
    line.set_ydata(ir[frame,...])
    im.set_array(spe[frame,...])
    text1.set_text(str(frame))
    text2.set_text(str(frame))
ani = animation.FuncAnimation(fig=fig, func=update, frames=ir.shape[0], interval=100)
display(ani)
ani.save('animation.gif', writer='imagemagick', fps=10)
plt.close(fig)

,AWARE STUDY ID:,Calculated age (years):,Sex:,Race/ethnicity: (choice=White),Race/ethnicity: (choice=Black / African American),Race/ethnicity: (choice=Hispanic / Latino),Race/ethnicity: (choice=Asian),Race/ethnicity: (choice=Other),Height (cm):,Weight (kg):,...,Post FEF2575 (%pred):,BDR (as percent of baseline FEV1),BDR (as percent of predicted FEV1),BDR (liters),BDR (difference in %preds),Test Number,Post-BD,Spectrogram,IR,isExhale
0,1,16.5,Female,Unchecked,Checked,Unchecked,Unchecked,Unchecked,160.2,75.8,...,NaN,NaN,NaN,NaN,NaN,1,0.0,"[[[0.4678940160326147, 0.6033473159926418, 0.5...","[[0.21588277063305553, 1.1021004976163198, 1.8...",False
1,1,16.5,Female,Unchecked,Checked,Unchecked,Unchecked,Unchecked,160.2,75.8,...,NaN,NaN,NaN,NaN,NaN,1,0.0,"[[[0.4938896698862547, 0.5178164977516541, 0.4...","[[0.04251350715381178, 0.19969075104220993, 0....",True
2,1,16.5,Female,Unchecked,Checked,Unchecked,Unchecked,Unchecked,160.2,75.8,...,NaN,NaN,NaN,NaN,NaN,1,0.0,"[[[0.30247918783415195, 0.3960567787503717, 0....","[[0.04257721866942041, 0.1982546199046874, 0.3...",False
3,1,16.5,Female,Unchecked,Checked,Unchecked,Unchecked,Unchecked,160.2,75.8,...,NaN,NaN,NaN,NaN,NaN,1,0.0,"[[[0.47066780197986435, 0.5718039926225023, 0....","[[0.032875488166469, 0.1331835845375863, 0.178...",True
4,1,16.5,Female,Unchecked,Checked,Unchecked,Unchecked,Unchecked,160.2,75.8,...,NaN,NaN,NaN,NaN,NaN,1,0.0,"[[[0.4880630818473849, 0.301699291081198, 0.38...","[[-0.0018537068641512418, 0.09481410239011742,...",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,28,38.7,Male,Checked,Unchecked,Unchecked,Unchecked,Unchecked,177.8,87.5,...,NaN,NaN,NaN,NaN,NaN,2,0.0,"[[[0.5596518805411305, 0.5658454016977607, 0.5...","[[0.023734086787247057, 0.10871619431012623, 0...",True
602,28,38.7,Male,Checked,Unchecked,Unchecked,Unchecked,Unchecked,177.8,87.5,...,NaN,NaN,NaN,NaN,NaN,2,0.0,"[[[0.4086370473294051, 0.40713564199379093, 0....","[[0.02748109963612492, 0.13656964691688023, 0....",False
603,28,38.7,Male,Checked,Unchecked,Unchecked,Unchecked,Unchecked,177.8,87.5,...,NaN,NaN,NaN,NaN,NaN,2,0.0,"[[[0.567963241462616, 0.5728285143877698, 0.57...","[[0.016163154864489087, 0.07046094588348148, 0...",True
604,28,38.7,Male,Checked,Unchecked,Unchecked,Unchecked,Unchecked,177.8,87.5,...,NaN,NaN,NaN,NaN,NaN,2,0.0,"[[[0.5183902025896288, 0.525548579684853, 0.54...","[[0.021099878606450337, 0.09874758456086537, 0...",False


AWARE STUDY ID:                                                                                      1
Calculated age (years):                                                                           16.5
Sex:                                                                                            Female
Race/ethnicity: (choice=White)                                                               Unchecked
Race/ethnicity: (choice=Black / African American)                                              Checked
Race/ethnicity: (choice=Hispanic / Latino)                                                   Unchecked
Race/ethnicity: (choice=Asian)                                                               Unchecked
Race/ethnicity: (choice=Other)                                                               Unchecked
Height (cm):                                                                                     160.2
Weight (kg):                                                             

(25, 960)
torch.Size([25, 224, 224])


MovieWriter imagemagick unavailable; using Pillow instead.
